# proto3 - started 2019-03-31

* 2019-03-31 created python 3.7 virtual environment, py37
* updated Anaconda and various other packages into py37
* now, more experiments with pinboard
    * pinboard library here: https://pypi.org/project/pinboard/
    * and on github here: https://github.com/lionheart/pinboard.py


> Having completed "API Acquaintence" in proto1, aim is to

* [x] understand data structure for entries
* [x] understand data structure for tags
* [x] _develop and test_ approaches to atomizing tags
    * done! herein

In [187]:
!python -V

Python 3.7.3


In [188]:
import sys
import urllib

In [189]:
import pinboard
from ruamel.yaml import YAML
from time import sleep
from random import sample, random
import pandas as pd

In [190]:
yaml=YAML(typ='safe')  

In [211]:
with open("data/keys/pbkey.yaml") as kh:
    fin = kh.read()
    config = yaml.load(fin)

In [212]:
pb = pinboard.Pinboard(config.get('uname') + ":" + config.get("key"))
type(pb)

pinboard.pinboard.Pinboard

## figure out TAGS data structure

In [193]:
tags_pb = pb.tags.get()
print("type of tags_pb is", type(tags_pb))

type of tags_pb is <class 'list'>


In [194]:
print("type of an item of tags_pb is", type(tags_pb[0]))

type of an item of tags_pb is <class 'pinboard.pinboard.Tag'>


In [195]:
# how many tags do I have?
len(tags_pb)

2128

In [196]:
print("type of an item.name of tags_pb is", type(tags_pb[0].name))

type of an item.name of tags_pb is <class 'str'>


In [197]:
tags_df = pd.DataFrame([[_.name, _.count] for _ in tags_pb], columns=["tag_in", "count"])

In [198]:
tags_df.head()

,tag_in,count
0,$chuckable,42
1,$chuckable(2016),3
2,$read-often,5
3,$rr,54
4,&advice,2


In [199]:
tags_df.to_excel("./data/tags.xlsx")

## make some fixes

In [ ]:
# rename tags
# pb.tags.rename(old='ppython', new='python')

In [24]:
pb.tags.rename(old="coms", new="_coms")

{'result': 'done'}

In [111]:
replacements = [
    ("app", "_app"), ("math", "_math"), ("dmml", "_dmml"), ("_dmm", "_dmml"),
    ("econ", "_econ"), ("eco", "_econ"), ("muse", "_muse"), ("survival.analysis", "survival-analysis")
]

In [112]:
for r in replacements:
    o = r[0]
    n = r[1]
    print("replacing ", o, " with ", n)
    pb.tags.rename(old=o, new=n)

replacing  app  with  _app
replacing  math  with  _math
replacing  dmml  with  _dmml
replacing  _dmm  with  _dmml
replacing  econ  with  _econ
replacing  eco  with  _econ
replacing  muse  with  _muse
replacing  survival.analysis  with  survival-analysis


## replace underscores with dashes

I have 499 tags with "-" as proxy for space, and 100 with "_".  
Therefore, replace all underscores with hyphens.  
UNLESS it is a leading underscore, in which case, skip it

In [213]:
tags_pb = pb.tags.get()
tags_df = pd.DataFrame([[_.name, _.count] for _ in tags_pb], columns=["tag_in", "count"])

In [219]:
for t in tags_df["tag_in"]:
    if t[0] == "_":
        pass
    elif "_" in t:
        new_t = t.replace("_","-")
        print(t, "\t -> ", new_t)
        pb.tags.rename(old=t, new=new_t)

&cheat_sheet 	 ->  &cheat-sheet
&op_inst 	 ->  &op-inst
@author(matt_might) 	 ->  @author(matt-might)
@author(peter_norvig) 	 ->  @author(peter-norvig)
Disk_Utility 	 ->  Disk-Utility
Greg_Wilson 	 ->  Greg-Wilson
Mac_OS 	 ->  Mac-OS
alexander_technique 	 ->  alexander-technique
anomaly_detection 	 ->  anomaly-detection
barry_ritholtz 	 ->  barry-ritholtz
base_graphics 	 ->  base-graphics
base_r 	 ->  base-r
bean_soup 	 ->  bean-soup
benjamin_franklin 	 ->  benjamin-franklin
big_numbers 	 ->  big-numbers
board_games 	 ->  board-games
bump_chart 	 ->  bump-chart
burger_king 	 ->  burger-king
cameron_crowe 	 ->  cameron-crowe
capital_allocation 	 ->  capital-allocation
capital_markets 	 ->  capital-markets
cheat_sheet 	 ->  cheat-sheet
clip_art 	 ->  clip-art
corn_bread 	 ->  corn-bread
corp_fin 	 ->  corp-fin
corporate_finance 	 ->  corporate-finance
curtis_mayfield 	 ->  curtis-mayfield
customer_analytics 	 ->  customer-analytics
data_structure 	 ->  data-structure
desktop_environment 

## fix the &amps

In [204]:
tags_pb = pb.tags.get()
tags_df = pd.DataFrame([[_.name, _.count] for _ in tags_pb], columns=["tag_in", "count"])

In [205]:
ampamps = [_ for _ in tags_df["tag_in"] if "&amp;amp;" in _]
print(len(ampamps))

0


In [46]:
for a in ampamps:
    fixed = "&" + a[9:]
    print(a, fixed)
    pb.tags.rename(old=a, new=fixed)

&amp;amp;advice &advice
&amp;amp;article &article
&amp;amp;blog &blog
&amp;amp;book &book
&amp;amp;book-review &book-review
&amp;amp;cheat-sheets &cheat-sheets
&amp;amp;class &class
&amp;amp;course &course
&amp;amp;documentation &documentation
&amp;amp;explainer &explainer
&amp;amp;guide &guide
&amp;amp;how-to &how-to
&amp;amp;inspiration &inspiration
&amp;amp;list &list
&amp;amp;long-article &long-article
&amp;amp;manual &manual
&amp;amp;merchant &merchant
&amp;amp;op_inst &op_inst
&amp;amp;paper &paper
&amp;amp;pdf &pdf
&amp;amp;prank &prank
&amp;amp;present &present
&amp;amp;presentation &presentation
&amp;amp;primer &primer
&amp;amp;project &project
&amp;amp;r-package &r-package
&amp;amp;reading-list &reading-list
&amp;amp;recipe &recipe
&amp;amp;reference &reference
&amp;amp;repo &repo
&amp;amp;review &review
&amp;amp;share &share
&amp;amp;syllabus &syllabus
&amp;amp;textbook &textbook
&amp;amp;tip &tip
&amp;amp;tips &tips
&amp;amp;tool &tool
&amp;amp;tutorial &tutorial
&amp;amp;u

In [182]:
# refresh the tag list
tags_pb_2 = pb.tags.get()
tags_df_2 = pd.DataFrame([[_.name, _.count] for _ in tags_pb_2], columns=["tag_in", "count"])

In [183]:
tags_df_2.head()

,tag_in,count
0,$chuckable,42
1,$chuckable(2016),3
2,$read-often,5
3,$rr,54
4,&advice,2


In [184]:
amps = [_ for _ in tags_df_2["tag_in"] if _[0:4] == "&amp"]
print(len(amps))
print(amps)

1
['&amp;tutorial']


In [185]:
for a in amps:
    fixed = "&" + a[5:]
    print(a, fixed)
    pb.tags.rename(old=a, new=fixed)

&amp;tutorial &tutorial


# split the dots - aka atomize joined tags


In [186]:
posts = pb.posts.all()

In [171]:
print(type(posts), len(posts))

<class 'list'> 2600


In [172]:
for p in posts:
    tags = p.tags
    dotty_tags = 0
    for t in tags:
        if "." in t:
            dotty_tags += 1
    if dotty_tags > 0:
        new_tags = tags.copy()
        # print("original tags list:")
        print(tags)
        for t in new_tags:
            if "." in t:
                if t == 'parse.ly':
                    pass
                else:
                    split_list = t.split(".")
                    new_tags.remove(t)
                    new_tags.extend(split_list)
        # print("new tags list:")
        final_tags = list(set(new_tags))
        print(final_tags)
        print(p)
        print("\n)")
        p.tags = final_tags
        p.save()

['parse.ly', '_data', 'streaming', 'logging', '_dmml', 'engineering']
['logging', '_data', 'parse.ly', '_dmml', 'streaming', 'engineering']
<Bookmark description="Real-time Streams & Logs — streams 0.1 documentation" url="www.parse.ly">

)
['_coms.lang.python.jupyter', 'interactive', '_dmml', 'reactive', '_coms', 'ipython-notebook', 'python', 'lang']
['_dmml', '_coms', 'ipython-notebook', 'reactive', 'python', 'interactive', 'lang', 'jupyter']
<Bookmark description="Building Interactive Dashboards with Jupyter" url="blog.dominodatalab.com">

)
['_sci.astro.mars', '_sci.astro.orery', '_coms', 'astro', 'orery', '_sci', 'project']
['_coms', 'astro', '_sci.astro.orery', 'orery', 'mars', '_sci', 'project']
<Bookmark description="How far away is Mars?" url="feeds.boingboing.net">

)
['information-theory', '_stat.covariance', 'entropy', '@curriculum', '_stat', 'corr']
['_stat', 'entropy', '@curriculum', 'corr', 'information-theory', 'covariance']
<Bookmark description="A Beginner’s Guide to E

## Sidebar on removing and adding items to lists

In [106]:
l = ["apples", "pears", "fish", "nuggets"]

In [107]:
l.remove("pears")

In [108]:
l

['apples', 'fish', 'nuggets']

In [109]:
l.append("swine")

In [110]:
l

['apples', 'fish', 'nuggets', 'swine']

-----
# Junk beneath

In [21]:
len([_ for _ in tags_df["tag_in"] if _[0]=="&"])

102

In [15]:
tags_df["dot_check"] = tags_df["tag_in"].str.contains("\.")

In [72]:
tags_df[tags_df["dot_check"]==True]

,tag_in,count,dot_check
101,.someday,1,True
217,_app.firefox,1,True
218,_app.pythonista.scene,1,True
219,_app.spark,1,True
220,_app.terminal,1,True
221,_app.video-editor,1,True
224,_coms.api,1,True
225,_coms.best-practices,3,True
226,_coms.cluster,1,True
227,_coms.documentation,1,True


In [133]:
def dump_amps(strin):
    """remove all variations of &amp;"""
    evil = ["&amp;", "amp;"]
    for e in evil:
        strin = strin.replace(e, "")
    return strin
assert(dump_amps("&amp;")=="")
assert(dump_amps("&amp;amp;")=="")

In [134]:
def tag_check(strin):
    """split any tag containing a dot"""
    if strin[-3:] == ".js":
        candidates = strin
    else:
        candidates = [dump_amps(_) for _ in strin.split(".") if len(_) > 0]
    results = candidates
    return results

In [135]:
# tags_df["tag_splits"] = tags_df["tag_in"].str.split(".")
tags_df["tag_splits"] = tags_df["tag_in"].apply(tag_check)


In [136]:
tags_df[tags_df["dot_check"]==True]

,tag_in,count,dot_check,tag_splits
101,.someday,1,True,[someday]
217,_app.firefox,1,True,"[_app, firefox]"
218,_app.pythonista.scene,1,True,"[_app, pythonista, scene]"
219,_app.spark,1,True,"[_app, spark]"
220,_app.terminal,1,True,"[_app, terminal]"
221,_app.video-editor,1,True,"[_app, video-editor]"
224,_coms.api,1,True,"[_coms, api]"
225,_coms.best-practices,3,True,"[_coms, best-practices]"
226,_coms.cluster,1,True,"[_coms, cluster]"
227,_coms.documentation,1,True,"[_coms, documentation]"


In [137]:
tags_df

,tag_in,count,dot_check,tag_splits
0,$chuckable,66,False,[$chuckable]
1,$chuckable(2016),3,False,[$chuckable(2016)]
2,$read-often,4,False,[$read-often]
3,$rr,50,False,[$rr]
4,&amp;,1,False,[]
5,&amp;amp;advice,2,False,[advice]
6,&amp;amp;article,5,False,[article]
7,&amp;amp;blog,2,False,[blog]
8,&amp;amp;book,2,False,[book]
9,&amp;amp;book-review,1,False,[book-review]


In [139]:
tags_df.to_csv("./data/tags_cleaned.csv")

## figure out POSTS data structue

In [7]:
posts_pb = pb.posts.all()
type(posts_pb)

list